# Galerkin, BDF2

In [4]:
from dolfin import *
import numpy as np
import sympy as sym
import csv

# ------------------------------------------ #

def compute_errors(u_e, u, t, mesh):
    L2n = errornorm(u_e, u, norm_type='L2', degree_rise=3, mesh=mesh)
    H1n = errornorm(u_e, u, norm_type='H1', degree_rise=3, mesh=mesh)
    return L2n, H1n

def compute_extrema(u, t):
    maxval = np.amax(u.vector().get_local())
    minval = np.amin(u.vector().get_local())
    return maxval, minval

def evolve_t(folder, nx, dt, T, degree, u_code, f_code, sigma, mu, velocity):
    t = 0.0
    u_exact = Expression(u_code, degree = degree+1, t = t)
    f = Expression(f_code, degree = degree+1, t = t)

    mesh = UnitSquareMesh(nx,nx)
    Q = FunctionSpace(mesh, "CG", degree)

    # Set up boundary condition
    u_D = Expression(u_exact.cppcode, degree = degree+1, t = t)
    
    def boundary(x, on_boundary):
        return on_boundary

    # Test and trial functions
    u, v = TrialFunction(Q), TestFunction(Q)
    u_n0 = interpolate(u_D, Q)
    u_n1 = Function(Q)
    u_ = Function(Q)

        # Create progress bar
    progress = Progress('Time-stepping')
    set_log_level(PROGRESS)
    
# -------need u_n1------------------------------- #

    # Galerkin variational problem
    F = (u - u_n0)*v*dx
    F += dt*(mu*dot(grad(v), grad(u))*dx + v*dot(velocity, grad(u))*dx + sigma*v*u*dx - f*v*dx)

    # Create bilinear and linear forms
    a1 = lhs(F)
    L = rhs(F)

    # Assemble matrices
    A1 = assemble(a1)

    t += dt
    
    u_.rename('u','u')
    
    u_D.t = t
    f.t = t
    velocity.t = t 
    u_exact.t = t
    

    b = assemble(L)
#     bc = DirichletBC(Q, u_D, boundary)
#     bc.apply(A1)
#     bc.apply(b)
    
    solve(A1, u_n1.vector(), b)#, 'gmres')
    progress.update(t / T)

# -------------------------------------- #

    # Galerkin variational problem
    F = (1.5*u - 2.0*u_n1 + 0.5*u_n0)*v*dx
    F += dt*(mu*dot(grad(v), grad(u))*dx + v*dot(velocity, grad(u))*dx + sigma*v*u*dx - f*v*dx)

    # Create bilinear and linear forms
    a1 = lhs(F)
    L = rhs(F)



    # Outputting files
    out_file_uexact = File(folder+"Exact_u_"+str(nx)+".pvd") 
    out_file_ubar = File(folder+"Galerk_u_"+str(nx)+".pvd") 
    ue = interpolate(u_exact, Q)
    
    u_.rename('u','u')
    
    # Save t = 0.0
    # #out_file_uexact << (ue, float(t))
    # out_file_ubar << (u_, float(t))

    while t - T + dt < DOLFIN_EPS:
        # Step 1 Solve on Coarse Grid
        t += dt
        
        u_.rename('u','u')
        
        u_D.t = t
        f.t = t
        u_exact.t = t
        velocity.t = t
        
        # Assemble matrices
        A1 = assemble(a1)

        b = assemble(L)
#         bc = DirichletBC(Q, u_D, boundary)
#         bc.apply(A1)
#         bc.apply(b)
        
        solve(A1, u_.vector(), b)#, 'gmres')
        progress.update(t / T)

        out_file_ubar << (u_, float(t))
        u_n0.assign(u_n1)
        u_n1.assign(u_)

    #out_file_ubar << (u_, float(t))
    L2, H1 = compute_errors(u_exact, u_, t, mesh)
    maxval, minval = compute_extrema(u_, t)
    
    print nx, L2, H1,maxval,minval
    return str(L2)+','+str(H1)+','+str(maxval)+','+str(minval)


# ------------------------------------------ #

from datetime import datetime, date

def run(nx,dt,sigma,mu):
    d_date = datetime.now()
    today = d_date.strftime('%Y-%m-%d')
    f = open(today+'_log.txt',"a+")


    T = 6.0
    degree = 2


    #sigma = 0.1
    #mu = 0.005
    
    folder = 'Galerkin/sigma'+str(sigma)+'_mu'+str(mu)
    velocity = Expression(('cos(t)','sin(t)'), degree = degree, t = 0)
    adr_f = Expression('exp(-(pow(x[0]-0.5,2)+pow(x[1]-0.5,2))/pow(0.07,2))', degree = degree)
    
    sim_data = 'E, dt='+str(dt)+', T='+str(T)+', P'+str(degree)
    f.write(sim_data+'\n')


    now = d_date.strftime("%I:%M:%S %p")
    f.write('start: '+now+'\n\n')
    
    output = evolve_t(folder, nx, dt, T, degree, '0', adr_f.cppcode, sigma, mu, velocity)
    f.write(str(nx)+','+str(dt)+','+output+'\n')


    d_date = datetime.now()
    now = d_date.strftime("%I:%M:%S %p")
    f.write('\nend: '+now+'\n\n')
    f.write(notes)

    f.close()

In [24]:
dt = 0.1
nx = 100
sigma = 0.1
mu = 0.005
run(nx,dt,sigma,mu)
sigma = 0.1
mu = 0.0005
run(nx,dt,sigma,mu)
sigma = 0.1
mu = 0.00005
run(nx,dt,sigma,mu)

100 0.0224556047838 0.377730061343 0.101166293744 -0.00187857853312
100 0.0262106653817 1.17764966335 0.118766919246 -0.0103466612513
100 0.0270994865303 3.56846973947 0.161511167385 -0.0549452635698


In [25]:
sigma = 0.1
mu = 0.000005
run(nx,dt,sigma,mu)

100 0.0286860184442 10.017557806 0.172332049644 -0.128969829502


In [5]:
dt = 0.1
sigma = 0.1
mu = 0.00005
run(25,dt,sigma,mu)
run(50,dt,sigma,mu)
run(100,dt,sigma,mu)
run(200,dt,sigma,mu)
run(400,dt,sigma,mu)

25 5.00471459475e+35 8.20899497021e+36 3.70465381237e+35 -7.41624278401e+36
50 7.84019439091e+77 1.5434054514e+79 4.08602027008e+77 -1.44054845955e+79
100 1.95300866111e+81 2.40981384215e+83 6.62224745077e+82 -8.64234342698e+82


KeyboardInterrupt: 

400 0.0270457625692 11.4126414057 0.168801097185 -0.0370192307478
